In [1]:
# If you're on Colab, set Runtime -> Change runtime type -> GPU (T4, A100, etc.)
!nvidia-smi



/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip -q install --upgrade pip
!pip -q install opencv-python pillow matplotlib albumentations tqdm einops

# (Colab ships with torch/torchvision; uncomment if you need a specific CUDA build)
# !pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu121

# Clone official MODNet
%cd /content
!git clone https://github.com/ZHKKKe/MODNet.git
%cd MODNet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00
/content
Cloning into 'MODNet'...
remote: Enumerating objects: 276, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 276 (delta 21), reused 16 (delta 16), pack-reused 236 (from 1)
Receiving objects: 100% (276/276), 60.77 MiB | 21.30 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/content/MODNet


In [7]:
import os, sys, glob, math, random, time
from pathlib import Path
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [10]:
import os, sys, types, importlib.util

# --- 0) Locate repo layout ---
CANDIDATES = [
    "/content/MODNet/src",            # typical after: git clone && cd MODNet
    "/content/src",                   # if you moved only src/
]
ROOT = next((p for p in CANDIDATES if os.path.isdir(p)), None)
assert ROOT, "Couldn't find MODNet/src. Make sure you've cloned the repo to /content/MODNet."

MODELS_DIR = os.path.join(ROOT, "models")
MODNET_PY  = os.path.join(MODELS_DIR, "modnet.py")
assert os.path.isfile(MODNET_PY), f"modnet.py not found at {MODNET_PY}"


In [13]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import os

# CHANGE THIS to your folder in Drive
DATA_ROOT = Path('/content/drive/MyDrive/modnet_data')

# Define all subdirectories
TRAIN_IMG_DIR = DATA_ROOT/'train/images'
TRAIN_MAT_DIR = DATA_ROOT/'train/mattes'
VAL_IMG_DIR   = DATA_ROOT/'val/images'
VAL_MAT_DIR   = DATA_ROOT/'val/mattes'
TEST_IMG_DIR  = DATA_ROOT/'test/images'
BG_DIR        = DATA_ROOT/'bgs'           # optional backgrounds

# Function to create dirs if not exist
def ensure_dir(path):
    if not path.exists():
        print(f"No folder {path}, creating...")
        path.mkdir(parents=True, exist_ok=True)
        print("✔ New directory created.")
    else:
        print(f"{path} exists, do nothing.")

# Ensure all needed directories
for d in [TRAIN_IMG_DIR, TRAIN_MAT_DIR, VAL_IMG_DIR, VAL_MAT_DIR, TEST_IMG_DIR, BG_DIR]:
    ensure_dir(d)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
No folder /content/drive/MyDrive/modnet_data/train/images, creating...
✔ New directory created.
No folder /content/drive/MyDrive/modnet_data/train/mattes, creating...
✔ New directory created.
No folder /content/drive/MyDrive/modnet_data/val/images, creating...
✔ New directory created.
No folder /content/drive/MyDrive/modnet_data/val/mattes, creating...
✔ New directory created.
No folder /content/drive/MyDrive/modnet_data/test/images, creating...
✔ New directory created.
No folder /content/drive/MyDrive/modnet_data/bgs, creating...
✔ New directory created.
